In [1]:
# Install the required package if you haven't already
install.packages("bigrquery")

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)



In [7]:
# Install the magrittr package
install.packages("magrittr")

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)



In [9]:
# Install the dplyr package
install.packages("dplyr")

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)



In [10]:
# Load the required packages
library(bigrquery)
library(magrittr)
library(dplyr)
# The %>% operator is part of the magrittr package, which provides a mechanism for chaining commands together in a way that is easy
# to read and understand.
# Set up your project ID, dataset ID, table name, and search parameters
project_id <- "yhcr-prd-phm-bia-core"
dataset_id <- "CB_FDM_BradfordRoyalInfirmary"
table_name <- "tbl_pharmacy"
search_column <- "drug_description"
search_phrase <- "prednisolone"

# Define the function to search the database
search_database <- function(project_id, dataset_id, table_name, search_column, search_phrase) {
  tryCatch({
    # Construct the SQL query
    query <- paste0("
      SELECT *
      FROM `", project_id, ".", dataset_id, ".", table_name, "`
      WHERE ", search_column, " LIKE '%", search_phrase, "%';
    ")

    # Execute the query and return the result as a data frame
    result <- bigrquery::bq_project_query(project_id, query)
    # Retrieve the data as a data frame
    result_df <- result %>% collect()
    # Extract the relevant columns from the result
    relevant_columns <- colnames(result_df) %in% search_column
    return(result_df[, relevant_columns])
  },
  error = function(e) {
    print(paste0("An error occurred: ", e))
  })
}

# Call the function with the search parameters
df <- search_database(project_id, dataset_id, table_name, search_column, search_phrase)

# Print the first few rows of the result
print(head(df))



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




[1] "An error occurred: Error in UseMethod(\"collect\"): no applicable method for 'collect' applied to an object of class \"bq_table\"\n"
[1] "An error occurred: Error in UseMethod(\"collect\"): no applicable method for 'collect' applied to an object of class \"bq_table\"\n"


In [ ]:
# The errors I encountered above is because the collect() function from the dplyr package is not compatible with 
# the bq_table object returned by the bigrquery package.

# To retrieve the data from a BigQuery query as a data frame, you can use the bq_table_download() function from the bigrquery package. This function downloads the data from the BigQuery table to a local data frame.

# the updated code with the bq_table_download() function is below:

In [11]:
# Load the required packages
library(bigrquery)

# Set up your project ID, dataset ID, table name, and search parameters
project_id <- "yhcr-prd-phm-bia-core"
dataset_id <- "CB_FDM_BradfordRoyalInfirmary"
table_name <- "tbl_pharmacy"
search_column <- "drug_description"
search_phrase <- "prednisolone"

# Define the function to search the database
search_database <- function(project_id, dataset_id, table_name, search_column, search_phrase) {
  tryCatch({
    # Construct the SQL query
    query <- paste0("
      SELECT *
      FROM `", project_id, ".", dataset_id, ".", table_name, "`
      WHERE ", search_column, " LIKE '%", search_phrase, "%';
    ")

    # Execute the query and return the result as a data frame
    result <- bigrquery::bq_project_query(project_id, query)
    # Download the data as a data frame
    result_df <- bigrquery::bq_table_download(result)
    # Extract the relevant columns from the result
    relevant_columns <- colnames(result_df) %in% search_column
    return(result_df[, relevant_columns])
  },
  error = function(e) {
    print(paste0("An error occurred: ", e))
  })
}

# Call the function with the search parameters
df <- search_database(project_id, dataset_id, table_name, search_column, search_phrase)

# Print the first few rows of the result
print(head(df))


# A tibble: 6 × 1
  drug_description                                    
  <chr>                                               
1 Methylprednisolone IV Infusion - adults (500mg vial)
2 Methylprednisolone IV Infusion - adults (500mg vial)
3 Methylprednisolone IV Infusion - adults (500mg vial)
4 Methylprednisolone IV Infusion - adults (1g vial)   
5 Methylprednisolone IV Infusion - adults (500mg vial)
6 Methylprednisolone IV Infusion - adults (500mg vial)
